<a href="https://colab.research.google.com/github/yoursdream123/PIAICBasicAssignments/blob/main/RAGbyChatGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Install necessary packages
!pip install langchain openai pinecone-client fastapi uvicorn nest-asyncio requests

# Import required modules
import os
import requests
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.embeddings.base import Embeddings
from fastapi import FastAPI, HTTPException
from nest_asyncio import apply
from uvicorn import run
from pinecone import Pinecone, ServerlessSpec

# Set API keys
os.environ["GEMINI_API_KEY"] = "your-gemini-api-key"  # Replace with your Gemini API key
os.environ["PINECONE_API_KEY"] = "your-pinecone-api-key"  # Replace with your Pinecone API key

# Define custom embedding class for Google Gemini
class GeminiEmbeddings(Embeddings):
    def embed_text(self, text: str):
        api_url = "https://gemini.googleapis.com/v1beta/embeddings"  # Adjust endpoint if needed
        headers = {
            "Authorization": f"Bearer {os.getenv('GEMINI_API_KEY')}",
            "Content-Type": "application/json",
        }
        payload = {"text": text}
        response = requests.post(api_url, headers=headers, json=payload)

        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            raise ValueError(f"Gemini API error: {response.status_code} {response.text}")

# Initialize Pinecone client
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)

index_name = "rag-index"
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # Ensure it matches the embedding size from Gemini
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Define documents
raw_documents = [
    "The history of AI includes milestones such as the founding of the field in 1956, expert systems in the 1970s, and machine learning advances in the 1980s. Key contributors include Alan Turing, John McCarthy, and Marvin Minsky."
]

documents = [Document(page_content=doc) for doc in raw_documents]

# Create vectorstore
embeddings = GeminiEmbeddings()
vectorstore = LangChainPinecone(
    index=pc.Index(index_name),
    embedding=embeddings.embed_text,
    text_key="content",
)
vectorstore.add_documents(documents)

# Set up RetrievalQA chain
retriever = vectorstore.as_retriever()
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        You are a helpful assistant. Use the following context to answer the question accurately:
        Context: {context}
        Question: {question}
        Answer:
    """
)
qa_chain = RetrievalQA(retriever=retriever, prompt=prompt_template)

# Define FastAPI app
app = FastAPI()

@app.post("/ask")
async def ask_question(question: str):
    try:
        response = qa_chain.run(question)
        return {"question": question, "response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run FastAPI server
apply()
run("app:app", host="0.0.0.0", port=8000)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '45a3be1ef9dfcc0b7d5df287b9c34cf3', 'Date': 'Wed, 25 Dec 2024 19:09:16 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [1]:
!pip install langchain pinecone-client google-generativeai openai tqdm python-dotenv -q
!pip install langchain-google-genai -U -q
!pip install --upgrade langchain -q
!pip install langchain-community -U -q
!pip install -qU langchain-google-vertexai
!pip install google-generativeai --upgrade -q
!pip install langchain-google-vertexai --upgrade -q
!pip install google-cloud google-cloud-language google-auth google-auth-oauthlib google-auth-httplib2 -q
!pip install -U langchain-google-vertexai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.2 MB/s eta 0:00:00


In [2]:
# @title
# Write API keys to a .env file
with open(".env", "w") as f:
    f.write("pcsk_38FVpk_5CwM7zM5EXesWxTAC3e6qgttKyKRrweChovNwWHP8uipEDgJo1Gs3CHjDwurCc7=")
    f.write("PINECONE_ENVIRONMENT=us-east-1'\n")
    f.write("GOOGLE_API_KEY_1=\n")
print(".env file created successfully!")

.env file created successfully!


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.getenv("pcsk_38FVpk_5CwM7zM5EXesWxTAC3e6qgttKyKRrweChovNwWHP8uipEDgJo1Gs3CHjDwurCc7")
PINECONE_ENVIRONMENT = os.getenv("us-east-1")
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY_1")

In [6]:
import pinecone
from pinecone import Pinecone

# Instead of using pinecone.init(), create a Pinecone instance:
pc = Pinecone(api_key="pcsk_38FVpk_5CwM7zM5EXesWxTAC3e6qgttKyKRrweChovNwWHP8uipEDgJo1Gs3CHjDwurCc7", environment="us-east-1")

index_name = "rag-index"
# Call list_indexes() on the Pinecone instance 'pc'
if index_name not in pc.list_indexes().names():  # Use pc.list_indexes().names() to get the index names
    pinecone.create_index(index_name, dimension=768)

index = pc.Index(index_name)
# Now you can use 'pc' to interact with Pinecone:
print(pc.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'rag-index-el177wn.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'rag-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [18]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the documents, specifying the correct encoding
loader = TextLoader("/prompt.txt", encoding='latin-1')  # Try 'latin-1' or 'cp1252'
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

print(f"Split into {len(docs)} chunks.")

Split into 3 chunks.


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the documents, specifying the correct encoding
loader = TextLoader("/prompt.txt", encoding='latin-1')  # Try 'latin-1' or 'cp1252'
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

print(f"Split into {len(docs)} chunks.")
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
import time

# Initialize the embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", api_key="GOOGLE_API_KEY_1")

# Upsert vectors into Pinecone
from tqdm import tqdm

# Upsert vectors into Pinecone
for i, doc in enumerate(tqdm(docs)):
    vector = embeddings.embed_query(doc.page_content)
    index.upsert([(str(i), vector, {"text": doc.page_content})])
    time.sleep(2)  # Introduce a 1-second delay between requests
print("Vectors upserted into Pinecone.")

print("Documents uploaded to Pinecone.")

In [9]:
# Install necessary packages
!pip install langchain openai pinecone-client fastapi uvicorn nest-asyncio requests

# Import required modules
import os
import requests
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.embeddings.base import Embeddings
from fastapi import FastAPI, HTTPException
from nest_asyncio import apply
from uvicorn import run
from pinecone import Pinecone, ServerlessSpec
from langchain.llms.base import LLM
from typing import Optional
import requests

In [11]:
from langchain.llms.base import LLM
import requests
from typing import Optional

In [15]:
# Set API keys
os.environ["GEMINI_API_KEY"] = "GOOGLE_API_KEY_1"  # Replace with your Gemini API key
os.environ["PINECONE_API_KEY"] = "pcsk_38FVpk_5CwM7zM5EXesWxTAC3e6qgttKyKRrweChovNwWHP8uipEDgJo1Gs3CHjDwurCc7"  # Replace with your Pinecone API key

# Define custom embedding class for Google Gemini
class GeminiEmbeddings(Embeddings):
    def embed_text(self, text: str):
        api_url = "https://gemini.googleapis.com/v1beta/embeddings"  # Adjust endpoint if needed
        headers = {
            "Authorization": f"Bearer {os.getenv('GEMINI_API_KEY')}",
            "Content-Type": "application/json",
        }
        payload = {"text": text}
        response = requests.post(api_url, headers=headers, json=payload)

        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            raise ValueError(f"Gemini API error: {response.status_code} {response.text}")

# Initialize Pinecone client
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)

index_name = "rag-index"
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # Ensure it matches the embedding size from Gemini
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Define documents
raw_documents = [
    "The history of AI includes milestones such as the founding of the field in 1956, expert systems in the 1970s, and machine learning advances in the 1980s. Key contributors include Alan Turing, John McCarthy, and Marvin Minsky."
]

documents = [Document(page_content=doc) for doc in raw_documents]


# Create vectorstore
class GeminiEmbeddings(Embeddings):
    def embed_text(self, text: str):
        api_url = "https://gemini.googleapis.com/v1beta/embeddings"  # Adjust the endpoint if needed
        headers = {
            "Authorization": f"Bearer {os.getenv('GEMINI_API_KEY')}",
            "Content-Type": "application/json",
        }
        payload = {"text": text}
        response = requests.post(api_url, headers=headers, json=payload)

        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            raise ValueError(f"Gemini API error: {response.status_code} {response.text}")

    def embed_documents(self, texts: list):
        """
        Embeds a list of texts.
        """
        return [self.embed_text(text) for text in texts]

    def embed_query(self, text: str):
        """
        Embeds a single query text.
        """
        return self.embed_text(text)

# Ensure the GEMINI_API_KEY environment variable is set
os.environ["GEMINI_API_KEY"] = "GOOGLE_API_KEY_1"

# Initialize embeddings
embeddings = GeminiEmbeddings()

# Use embeddings in vectorstore creation
vectorstore = LangChainPinecone(
    index=pc.Index(index_name),
    embedding=embeddings,
    text_key="content"
)










# Create vectorstore
#embeddings = GeminiEmbeddings()
#vectorstore = LangChainPinecone(
#    index=pc.Index(index_name),
#    embedding=embeddings.embed_text,
#    text_key="content",
#)
#vectorstore.add_documents(documents)

# Set up RetrievalQA chain
retriever = vectorstore.as_retriever()
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        You are a helpful assistant. Use the following context to answer the question accurately:
        Context: {context}
        Question: {question}
        Answer:
    """
)
#qa_chain = RetrievalQA(retriever=retriever, prompt=prompt_template)

#   ABhi ABhi LAYA






# Gemini LLM Class
class GeminiLLM(LLM):
    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        api_key = "your_gemini_api_key_here"  # Add your Gemini API key here
        api_url = "https://gemini.googleapis.com/v1beta/generateText"  # Replace with the correct endpoint
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "prompt": prompt,
            "temperature": 0.7,
            "max_tokens": 150,
        }
        response = requests.post(api_url, headers=headers, json=payload)

        if response.status_code == 200:
            return response.json().get("text", "").strip()
        else:
            raise ValueError(f"Gemini API error: {response.status_code} {response.text}")

    @property
    def _llm_type(self) -> str:
        return "gemini"

# Initialize Gemini LLM
gemini_llm = GeminiLLM()

# Define the retrieval chain
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use the following context to answer the question:

    Context: {context}
    Question: {question}

    Answer:"""
)

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_llm,  # Pass the Gemini LLM object
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True,
)


# Abhi Abhi laya




#qa_chain = RetrievalQA.from_chain_type(
    #retriever=retriever,
    #chain_type="stuff",  # Choose a chain type, e.g., "stuff" or "map_reduce"
    #chain_type_kwargs={"prompt": prompt_template},
    #return_source_documents=True,  # Optional: to return source documents
#)





# Define FastAPI app
app = FastAPI()

@app.post("/ask")
async def ask_question(question: str):
    try:
        response = qa_chain.run(question)
        return {"question": question, "response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run FastAPI server
apply()
run("app:app", host="0.0.0.0", port=8000)

ERROR:    Error loading ASGI app. Could not import module "app".
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/importer.py", line 19, in import_from_string
    module = importlib.import_module(module_str)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'app'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/config.py", line 435, in load
    self.loaded_app = import_from_string(self.app)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/importer.py", line 24, in import_from_string
    raise ImportFromStringError(message.format(module_str=module_str))
uv

TypeError: object of type 'NoneType' has no len()